# Chapter 11: Training Deep Neural Networks

## 11.1 The Vanishing/Exploding Gradients Problems

### 11.1.1 Glorot and He Initialization

### 11.1.2 Nonsaturating Activation Functions

### 11.1.3 Batch Normalization

### 11.1.4 Gradient Clipping

## 11.2 Reusing Pretrained Layers

### 11.2.1 Transfer Learning with Keras

### 11.2.2 Unsupervised Pretraining

### 11.2.3 Pretraining on an Auxiliary Task

## 11.3 Faster Optimizers

### 11.3.1 Momentum Optimization

### 11.3.2 Nesterov Accelerated Gradient

### 11.3.3 AdaGrad

### 11.3.4 RMSProp

### 11.3.5 Adam and Nadam Optimization

### 11.3.6 Learning Rate Scheduling

## 11.4 Avoiding Overfitting Through Regularization

### 11.4.1 $\ell_1$ and $\ell_2$ Regularization

### 11.4.2 Dropout

### 11.4.3 Monte Carlo (MC) Dropout

### 11.4.4 Max-Norm Regularization

## 11.5 Summary and Practical Guidelines